# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [7]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [8]:
llm_model = "gpt-3.5-turbo"

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [10]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory() # 대화 메모리 버퍼 저장
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [11]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [12]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [9]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [13]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?


In [11]:
memory.load_memory_variables({}) # 기존 대화에서 기억하고 있는 메모리를 가져온다. 

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew."}

### 대화에 컨텍스트를 부여하기 - ConversationBufferMemory()
- 대부분 LLM 모델은 'stateless', 즉 컨텍스트를 모른다. 
- 따라서, langchain의 ConversationBufferMemory()와 같이, 기존 대화 내용을 모델에 전달해주는 역할이 필요하다. 


In [14]:
memory = ConversationBufferMemory()

# 강제로 메모리 로그를 저장할 수 있다. 
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

print(memory.buffer)

Human: Hi
AI: What's up


In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [16]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [17]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### 효율적으로 기존 Context를 저장하기
- 그러나, LLM model에 prompt로 기존 대화 로그를 너무 많이 전달하게 되면, 많은 비용이 발생한다. 
- 따라서, 좀 더 효율적으로 컨텍스트를 전달하기 위해서 langchain에서는 ConversationBufferWindowMemory()와 같은 다양한 기능을 제공한다. 


##### (1) ConversationBufferWindowMemory()
- 최근 k 개의 데이터만 본다! 

In [18]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) # 윈도우 사이즈 k=1로 설정    
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})   

memory.load_memory_variables({}) # 최근 1개의 히스토리만 저장한다. -> {'history': 'Human: Not much, just hanging\nAI: Cool'}

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [19]:
# 실제 모델 학습에 적용하기 
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory, # memory 에 넣어주면 된다. 
    verbose = False
)


In [21]:
# 예측하기 
print(conversation.predict(input="Hi, my name is Andrew"))
print(conversation.predict(input="What is 1+1?"))
print(conversation.predict(input="What is my name?")) # K=1로 설정해뒀기 때문에, 이름을 모름

Hello Andrew! It's nice to meet you. How can I assist you today?
1+1 equals 2. Is there anything else you would like to know?
I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?


## ConversationTokenBufferMemory
- 과거 대화 내용을 토큰 단위로 메모리에 기록함

In [22]:
# ! pip install tiktoken

  Using cached tiktoken-0.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached regex-2024.7.24-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached tiktoken-0.7.0-cp312-cp312-macosx_11_0_arm64.whl (906 kB)
Using cached regex-2024.7.24-cp312-cp312-macosx_11_0_arm64.whl (279 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [24]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [25]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory
- 다른 매모리 기억 방법과 달리 Conversation 내용을 요약해서 과거 대화 기록을 저장

In [27]:
from langchain.memory import ConversationSummaryBufferMemory

In [32]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=400) #max_token_limit: 몇 토큰까지 요약한 내용을 저장할 것인가?
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})
memory.load_memory_variables({}) # token limit 이 400일 경우 전체 대화 내용을 다포함하고 있음

In [37]:
# max_token_limit 이 100일 때 
memory1 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) #max_token_limit: 몇 토큰까지 요약한 내용을 저장할 것인가?
memory1.save_context({"input": "Hello"}, {"output": "What's up"})
memory1.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory1.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})
memory1.load_memory_variables({}) # 결과: 요약된 내용이 저장되어 있음.

{'history': "System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events."}

In [39]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory1,
    verbose=True
)

In [40]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the morning meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. For the lunch meeting with the customer interested in AI, a demo highlighting the capabilities of our AI technology and how it can benefit their specific needs would be impressive. It's important to tailor the demos to each audience to make a strong impact."

In [41]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.'}

Reminder: Download your notebook to you local computer to save your work.

In [ ]:
# Su

# Summary 
ConversationBufferMemory
• This memory allows for storing of messages and then extracts the messages in a variable.
ConversationBufferWindowMemory
• This memory keeps a list of the interactions of the conversation over time. It only uses the last K interactions.
ConversationTokenBufferMemory
• This memory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.
ConversationSummaryMemory
• This memory creates a summary of the conversation over time.

+ Entity 메모리도 활용할 수 있음. 